In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.preprocessing import PolynomialFeatures
from pathlib import Path
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class TwoInputMarkupEstimator:
    def __init__(self, polynomial_degree=3):
        """
        Inicializa o estimador de markup com dois insumos usando a metodologia DLW.
        
        O estimador implementa uma abordagem em dois estágios para lidar com
        a endogeneidade dos insumos variáveis. No primeiro estágio, recuperamos
        a produtividade não observada. No segundo, estimamos os parâmetros da
        função de produção controlando pela produtividade estimada.
        
        Args:
            polynomial_degree (int): Grau do polinômio usado para aproximar
                                   a função de controle da produtividade
        """
        self.polynomial_degree = polynomial_degree
        self.production_params = {}  # Parâmetros estimados por setor
        self.productivity = {}       # Produtividade recuperada por setor
        self.diagnostics = {}        # Diagnósticos das estimações
    
    def prepare_data(self, df):
        """
        Prepara os dados para estimação, realizando transformações necessárias
        e limpeza básica.
        
        O método:
        1. Calcula logaritmos das variáveis relevantes
        2. Computa shares dos insumos
        3. Remove observações inválidas
        4. Organiza dados em estrutura de painel
        
        Args:
            df (DataFrame): Dados brutos
            
        Returns:
            DataFrame: Dados preparados para estimação
        """
        df_prep = df.copy()
        print("Preparando dados para estimação...")
        
        # Variáveis essenciais para o modelo de dois insumos
        input_vars = [
            'variable_input',    # Insumo variável composto
            'capital',          # Capital fixo
            'receita'           # Receita para cálculo de shares
        ]
        
        # Tratamento de valores não-positivos
        for var in input_vars:
            df_prep[var] = pd.to_numeric(df_prep[var], errors='coerce')
            mask = df_prep[var] > 0
            invalid_count = (~mask).sum()
            
            if invalid_count > 0:
                print(f"  Removidas {invalid_count} observações com valores não-positivos em {var}")
                df_prep.loc[~mask, var] = np.nan
        
        # Computar shares dos insumos
        df_prep['share_variable_input'] = df_prep['variable_input'] / df_prep['receita']
        
        # Calcular logaritmos
        for var in input_vars:
            df_prep[f'log_{var}'] = np.log(df_prep[var])
        
        # Variáveis necessárias para observações válidas
        required_vars = [f'log_{var}' for var in input_vars] + ['share_variable_input']
        
        # Remover valores infinitos e NaN
        df_prep = df_prep.replace([np.inf, -np.inf], np.nan)
        
        # Acompanhar progresso da limpeza
        n_before = len(df_prep)
        df_prep = df_prep.dropna(subset=required_vars)
        n_after = len(df_prep)
        
        print(f"\nObservações válidas: {n_after}/{n_before} ({n_after/n_before*100:.1f}%)")
        
        return df_prep
    
    def first_stage_estimation(self, data, sector):
        """
        Implementa o primeiro estágio da estimação da função de produção,
        onde recuperamos a produtividade não observada.

        O método segue três passos principais:
        1. Estima uma forma reduzida que relaciona receita com insumos
        2. Recupera a produtividade como resíduo desta estimação
        3. Valida as estimativas usando vários critérios diagnósticos

        Args:
            data (DataFrame): Dados preparados do setor
            sector (str): Identificador do setor

        Returns:
            tuple: (produtividade estimada, modelo da forma reduzida)
        """
        try:
            print(f"\nPrimeiro estágio da estimação - Setor: {sector}")

            # 1. Definição das variáveis de estado (capital) e controle (insumo variável)
            # Esta distinção é crucial devido ao timing das decisões da firma
            state_var = ['log_capital']  
            control_var = ['log_variable_input']

            # Construímos os termos polinomiais para aproximar a função de controle
            # Incluímos interações entre as variáveis para capturar complementaridades
            input_vars = state_var + control_var
            poly = PolynomialFeatures(
                degree=self.polynomial_degree, 
                include_bias=True,
                interaction_only=False
            )

            # Transformamos as variáveis em polinômios
            X_poly = poly.fit_transform(data[input_vars])
            feature_names = poly.get_feature_names_out(input_vars)
            X_df = pd.DataFrame(X_poly, columns=feature_names, index=data.index)

            # Estimamos a forma reduzida usando a receita como variável dependente
            y = data['log_receita']
            reduced_form = sm.OLS(y, X_df).fit()

            # Recuperamos a produtividade como resíduo
            omega_hat = reduced_form.resid

            # Realizamos diagnósticos da estimação
            self._first_stage_diagnostics(reduced_form, omega_hat, data, sector)

            # Armazenamos resultados para uso posterior
            self.productivity[sector] = omega_hat

            print(f"R² da forma reduzida: {reduced_form.rsquared:.4f}")
            print(f"Número de termos significativos: {sum(reduced_form.pvalues < 0.05)}")

            return omega_hat, reduced_form

        except Exception as e:
            print(f"Erro no primeiro estágio: {str(e)}")
            return None, None


    def _first_stage_diagnostics(self, model, omega, data, sector):
        """
        Implementa diagnósticos detalhados do primeiro estágio da estimação.

        Verificamos:
        1. Qualidade do ajuste do modelo
        2. Propriedades da produtividade estimada
        3. Correlações com variáveis observáveis

        Args:
            model: Modelo estimado da forma reduzida
            omega: Produtividade estimada
            data: Dados do setor
            sector: Identificador do setor
        """
        diagnostics = {}

        # Diagnósticos do modelo
        diagnostics['model'] = {
            'r2': model.rsquared,
            'r2_adj': model.rsquared_adj,
            'aic': model.aic,
            'bic': model.bic,
            'f_pvalue': model.f_pvalue
        }

        # Diagnósticos da produtividade
        diagnostics['productivity'] = {
            'mean': np.mean(omega),
            'std': np.std(omega),
            'skew': stats.skew(omega),
            'kurtosis': stats.kurtosis(omega)
        }

        # Correlações com insumos
        for var in ['log_variable_input', 'log_capital']:
            corr = np.corrcoef(data[var], omega)[0,1]
            diagnostics['correlation_' + var] = corr

        # Armazenar diagnósticos
        self.diagnostics[sector] = {
            'first_stage': diagnostics
        }

        # Reportar principais resultados
        print("\nDiagnósticos do Primeiro Estágio:")
        print(f"Média da produtividade: {diagnostics['productivity']['mean']:.4f}")
        print(f"Desvio padrão: {diagnostics['productivity']['std']:.4f}")
        print(f"Correlação com insumo variável: {diagnostics['correlation_log_variable_input']:.4f}")
      
    def second_stage_estimation(self, data, sector, omega_hat):
        """
        Implementa o segundo estágio da estimação da função de produção usando
        variáveis instrumentais, seguindo De Loecker e Warzynski (2012).

        O método utiliza uma abordagem de variáveis instrumentais em dois estágios (2SLS)
        onde instrumentamos o insumo variável com suas defasagens, mantendo o capital
        como variável predeterminada.

        Args:
            data (DataFrame): Dados preparados do setor
            sector (str): Identificador do setor
            omega_hat (array): Produtividade estimada no primeiro estágio

        Returns:
            dict: Elasticidades estimadas e diagnósticos
        """
        try:
            print(f"\nSegundo estágio da estimação - Setor: {sector}")

            # Preparar variáveis para estimação
            y = data['log_receita']
            X = data[['log_variable_input', 'log_capital']]

            # Incluir produtividade estimada como controle
            X['omega_hat'] = omega_hat
            X = sm.add_constant(X)

            # Construir matriz de instrumentos
            Z = np.column_stack([
                data['log_capital'],          # Capital é predeterminado
                data['log_variable_input'].shift(),  # Insumo variável defasado
                omega_hat                         # Produtividade estimada
            ])

            # Remover observações com valores ausentes
            valid_obs = ~np.isnan(Z).any(axis=1)
            y = y[valid_obs]
            X = X[valid_obs]
            Z = Z[valid_obs]

            # Primeiro estágio do 2SLS
            first_stage = sm.OLS(X['log_variable_input'], Z).fit()
            X_hat = first_stage.predict()

            # Substituir valor original pelo valor previsto
            X_2sls = X.copy()
            X_2sls['log_variable_input'] = X_hat

            # Segundo estágio (# Neste passo, estamos implicitamente definindo e minimizando a seguinte função objetivo: Q(θ) = (y - X_2sls * θ)' * (y - X_2sls * θ), onde:
            #   - Q(θ) denota a função objetivo que buscamos minimizar com respeito aos parâmetros θ.
            #   - y representa o vetor (n x 1) de observações da variável dependente (receita).
            #   - X_2sls é a matriz (n x k) de variáveis explicativas (insumos).
            #   - θ é o vetor (k x 1) de parâmetros desconhecidos que desejamos estimar.
            #   - (y - X_2sls * θ) denota o vetor (n x 1) de resíduos do modelo.
            #   - (y - X_2sls * θ)' representa a transposição do vetor de resíduos, resultando em um vetor 
            #     linha (1 x n).
            second_stage = sm.OLS(y, X_2sls).fit()

            # Extrair elasticidades
            elasticities = {
                'variable_input': second_stage.params['log_variable_input'],
                'capital': second_stage.params['log_capital']
            }

            # Realizar diagnósticos
            self._second_stage_diagnostics(first_stage, second_stage, elasticities, data, sector)

            # Armazenar parâmetros estimados
            self.production_params[sector] = elasticities

            print(f"Elasticidade do insumo variável: {elasticities['variable_input']:.4f}")
            print(f"Elasticidade do capital: {elasticities['capital']:.4f}")

            return elasticities

        except Exception as e:
            print(f"Erro no segundo estágio: {str(e)}")
            return None

    def _second_stage_diagnostics(self, first_stage, second_stage, elasticities, data, sector):
        """
        Implementa diagnósticos detalhados para o segundo estágio da estimação.

        Analisa:
        1. Qualidade dos instrumentos (first stage F-stat)
        2. Testes de sobreidentificação
        3. Validade das elasticidades estimadas

        Args:
            first_stage: Resultados do primeiro estágio do 2SLS
            second_stage: Resultados do segundo estágio do 2SLS
            elasticities: Elasticidades estimadas
            data: Dados do setor
            sector: Identificador do setor
        """
        diag = {}

        # 1. Força dos instrumentos
        diag['first_stage_f'] = first_stage.fvalue
        diag['first_stage_r2'] = first_stage.rsquared

        # 2. Qualidade do ajuste do segundo estágio
        diag['second_stage_r2'] = second_stage.rsquared
        diag['second_stage_f'] = second_stage.fvalue

        # 3. Validação das elasticidades
        returns_to_scale = sum(elasticities.values())
        diag['returns_to_scale'] = returns_to_scale

        # Imprimir resultados
        print("\nDiagnósticos do Segundo Estágio:")
        print(f"F-stat primeiro estágio: {diag['first_stage_f']:.2f}")
        print(f"R² primeiro estágio: {diag['first_stage_r2']:.4f}")
        print(f"R² segundo estágio: {diag['second_stage_r2']:.4f}")
        print(f"Retornos de escala: {returns_to_scale:.4f}")

        # Armazenar diagnósticos
        self.diagnostics[sector]['second_stage'] = diag
    
    
    def estimate_production_function(self, df, sector):
        """
        Integra os dois estágios da estimação da função de produção e coordena
        o processo completo de estimação.

        Esta função:
        1. Coordena os dois estágios da estimação
        2. Valida os resultados intermediários
        3. Garante a consistência entre os estágios
        4. Armazena os resultados finais

        Args:
            df (DataFrame): Dados preparados
            sector (str): Identificador do setor

        Returns:
            dict: Resultados completos da estimação
        """
        try:
            # Selecionar dados do setor
            data = df[df['setor_ipa_7d'] == sector].copy()
            sector_name = data['nome_setor'].iloc[0]
            print(f"\nEstimando função de produção para setor: {sector_name}")

            # Primeiro estágio
            omega_hat, reduced_form = self.first_stage_estimation(data, sector)

            if omega_hat is None:
                print(f"Falha no primeiro estágio para setor {sector_name}")
                return None

            # Segundo estágio
            elasticities = self.second_stage_estimation(data, sector, omega_hat)

            if elasticities is None:
                print(f"Falha no segundo estágio para setor {sector_name}")
                return None

            # Armazenar resultados completos
            results = {
                'elasticities': elasticities,
                'productivity': omega_hat,
                'nome_setor': sector_name,
                'n_obs': len(data)
            }

            return results

        except Exception as e:
            print(f"Erro na estimação da função de produção: {str(e)}")
            return None

    def calculate_markups(self, df, sector):
        """
        Calcula markups usando os parâmetros estimados da função de produção
        seguindo a metodologia DLW.

        O método implementa:
        1. Cálculo dos markups usando a condição de primeira ordem da firma
        2. Correções para produtividade
        3. Tratamento de outliers

        Args:
            df (DataFrame): Dados preparados
            sector (str): Identificador do setor

        Returns:
            array: Markups estimados
        """
        try:
            if sector not in self.production_params:
                raise ValueError(f"Parâmetros não estimados para setor {sector}")

            data = df[df['setor_ipa_7d'] == sector].copy()

            # Recuperar elasticidade do insumo variável
            theta = self.production_params[sector]['variable_input']

            # Calcular share do insumo variável
            share = data['variable_input'] / data['receita']
            share = share[share > 0].values

            # Aplicar correção de produtividade
            correction = np.exp(self.productivity[sector])

            # Calcular markup
            markup = theta * (1/share) * correction

            # Tratar outliers
            markup = self._treat_outliers(markup)

            return markup

        except Exception as e:
            print(f"Erro no cálculo dos markups: {str(e)}")
            return np.array([])

    def _treat_outliers(self, series, method='all'):
        """
        Implementa tratamento robusto de outliers usando múltiplos métodos.

        O método pode usar:
        1. Método IQR (Interquartile Range)
        2. Z-score modificado
        3. Combinação de métodos

        Args:
            series: Array com valores a serem tratados
            method: Método de tratamento ('iqr', 'zscore', 'all')

        Returns:
            array: Série tratada
        """
        if not isinstance(series, np.ndarray):
            series = np.array(series)

        if method == 'all':
            masks = []

            # Método IQR
            Q1, Q3 = np.percentile(series, [25, 75])
            IQR = Q3 - Q1
            masks.append((series >= Q1 - 1.5*IQR) & (series <= Q3 + 1.5*IQR))

            # Z-score modificado
            median = np.median(series)
            mad = np.median(np.abs(series - median))
            z_mod = 0.6745 * (series - median) / mad
            masks.append(np.abs(z_mod) <= 3.5)

            # Combinar métodos
            final_mask = np.logical_and.reduce(masks)
            return series[final_mask]

        return series
    
    def sensitivity_analysis(self, df, sector, markup_base):
        """
        Realiza análise de sensibilidade das estimativas de markup.

        A análise inclui:
        1. Variação do grau do polinômio
        2. Diferentes métodos de tratamento de outliers
        3. Comparação entre especificações

        Args:
            df: DataFrame com dados preparados
            sector: Identificador do setor
            markup_base: Markups do caso base

        Returns:
            dict: Resultados da análise de sensibilidade
        """
        try:
            data = df[df['setor_ipa_7d'] == sector].copy()
            n_obs = len(data)

            sensitivity = {}

            # Testar diferentes graus de polinômio
            for degree in [2, 3, 4]:
                try:
                    estimator_alt = self.__class__(polynomial_degree=degree)
                    results_alt = estimator_alt.estimate_production_function(df, sector)

                    if results_alt is not None:
                        markups_alt = estimator_alt.calculate_markups(df, sector)

                        # Comparar com caso base
                        common_valid = ~np.isnan(markup_base) & ~np.isnan(markups_alt)

                        if np.sum(common_valid) > 0:
                            corr = np.corrcoef(
                                markup_base[common_valid],
                                markups_alt[common_valid]
                            )[0,1]

                            mean_diff = np.mean(markups_alt[common_valid] - 
                                              markup_base[common_valid])

                            sensitivity[degree] = {
                                'correlation': corr,
                                'mean_diff': mean_diff,
                                'n_valid': np.sum(common_valid)
                            }

                except Exception as e:
                    print(f"Erro na análise para grau {degree}: {str(e)}")
                    continue

            return sensitivity

        except Exception as e:
            print(f"Erro na análise de sensibilidade: {str(e)}")
            return None

    def calculate_summary_statistics(self, markups):
        """
        Calcula estatísticas descritivas dos markups com tratamento robusto
        para amostras pequenas.
        """
        try:
            markups = np.asarray(markups).flatten()
            markups = markups[np.isfinite(markups)]

            n_obs = len(markups)
            if n_obs == 0:
                return {
                    'mean': np.nan,
                    'median': np.nan,
                    'std': np.nan,
                    'p25': np.nan,
                    'p75': np.nan,
                    'min': np.nan,
                    'max': np.nan,
                    'n_obs': 0,
                    'skewness': np.nan
                }

            # Estatísticas básicas
            basic_stats = {
                'mean': float(np.mean(markups)),
                'median': float(np.median(markups)),
                'std': float(np.std(markups)),
                'p25': float(np.percentile(markups, 25)),
                'p75': float(np.percentile(markups, 75)),
                'min': float(np.min(markups)),
                'max': float(np.max(markups)),
                'n_obs': n_obs
            }

            # Tratamento robusto de assimetria
            if n_obs >= 8:
                # Usar teste de assimetria apenas com 8+ amostras
                try:
                    skewness, skew_pvalue = stats.skewtest(markups)
                    basic_stats['skewness'] = float(skewness)
                    basic_stats['skew_pvalue'] = float(skew_pvalue)
                except Exception as e:
                    print(f"Erro no teste de assimetria: {e}")
                    basic_stats['skewness'] = np.nan
                    basic_stats['skew_pvalue'] = np.nan
            else:
                # Para amostras pequenas, calcular assimetria de forma simples
                basic_stats['skewness'] = float((basic_stats['mean'] - basic_stats['median']) / 
                                                (basic_stats['std'] if basic_stats['std'] != 0 else 1))
                basic_stats['skew_pvalue'] = np.nan

            return basic_stats

        except Exception as e:
            print(f"Erro no cálculo das estatísticas descritivas: {e}")
            return None        
        
    def diagnostic_tests(self, df, sector):
        """
        Implementa testes diagnósticos abrangentes para avaliar a qualidade
        da estimação da função de produção e dos markups resultantes.

        Os testes são organizados em cinco categorias principais:
        1. Especificação do modelo
        2. Propriedades dos resíduos
        3. Qualidade dos instrumentos
        4. Multicolinearidade
        5. Comparação com benchmarks da literatura

        Args:
            df (DataFrame): Dados preparados
            sector (str): Identificador do setor

        Returns:
            dict: Resultados dos testes diagnósticos
        """
        try:
            print(f"\nRealizando testes diagnósticos para setor: {sector}")
            diagnostics = {}

            # Selecionar dados do setor
            data = df[df['setor_ipa_7d'] == sector].copy()

            # 1. Testes de Especificação
            spec_tests = self._specification_tests(data)
            diagnostics['specification'] = spec_tests
            print("\nTestes de Especificação:")
            print(f"R² Ajustado: {spec_tests['r2_adj']:.4f}")
            print(f"Teste F (p-valor): {spec_tests['f_pvalue']:.4f}")

            # 2. Análise de Resíduos
            resid_analysis = self._residual_analysis(data)
            diagnostics['residuals'] = resid_analysis
            print("\nAnálise de Resíduos:")
            print(f"Normalidade (p-valor): {resid_analysis['normality_pvalue']:.4f}")
            print(f"Autocorrelação: {resid_analysis['autocorr']:.4f}")

            # 3. Análise dos Instrumentos
            iv_tests = self._instrument_analysis(data)
            diagnostics['instruments'] = iv_tests
            print("\nQualidade dos Instrumentos:")
            print(f"Teste de Sargan (p-valor): {iv_tests['sargan_pvalue']:.4f}")
            print(f"F primeiro estágio: {iv_tests['first_stage_f']:.2f}")

            # 4. Análise de Multicolinearidade
            X = data[['log_variable_input', 'log_capital']]
            vif_data = self._calculate_vif(X)
            diagnostics['collinearity'] = vif_data
            print("\nDiagnóstico de Multicolinearidade:")
            print(vif_data)

            # 5. Comparação com Literatura
            benchmark_tests = self._benchmark_comparison(data)
            diagnostics['benchmarks'] = benchmark_tests
            print("\nComparação com Literatura:")
            print(f"Elasticidade do trabalho dentro do intervalo esperado: {benchmark_tests['labor_elasticity_ok']}")
            print(f"Retornos de escala próximos de 1: {benchmark_tests['returns_to_scale_ok']}")

            return diagnostics

        except Exception as e:
            print(f"Erro nos testes diagnósticos: {str(e)}")
            return None

    def _specification_tests(self, data):
        """
        Realiza testes detalhados da especificação do modelo.
        """
        tests = {}

        try:
            # Ajuste do modelo
            tests['r2_adj'] = self.model_results.rsquared_adj
            tests['f_pvalue'] = self.model_results.f_pvalue

            # Teste de Ramsey RESET
            reset_test = self._perform_reset_test(data)
            tests['reset_pvalue'] = reset_test

            # Teste de forma funcional
            tests['functional_form'] = self._test_functional_form(data)

            return tests

        except Exception as e:
            print(f"Erro nos testes de especificação: {str(e)}")
            return {}

    def _residual_analysis(self, data):
        """
        Analisa propriedades dos resíduos da estimação.
        """
        analysis = {}

        try:
            residuals = self.model_results.resid

            # Teste de normalidade
            _, normality_pvalue = stats.normaltest(residuals)
            analysis['normality_pvalue'] = normality_pvalue

            # Autocorrelação
            analysis['autocorr'] = np.corrcoef(residuals[:-1], residuals[1:])[0,1]

            # Heteroscedasticidade
            het_test = self._test_heteroskedasticity(residuals, data)
            analysis['heteroskedasticity_pvalue'] = het_test

            return analysis

        except Exception as e:
            print(f"Erro na análise de resíduos: {str(e)}")
            return {}

    def _instrument_analysis(self, data):
        """
        Avalia a qualidade dos instrumentos utilizados.
        """
        tests = {}

        try:
            # Teste de Sargan para sobreidentificação
            tests['sargan_pvalue'] = self._calculate_sargan_test()

            # Força dos instrumentos
            tests['first_stage_f'] = self._calculate_first_stage_f()

            # Teste de Wu-Hausman para endogeneidade
            tests['hausman_pvalue'] = self._perform_hausman_test()

            return tests

        except Exception as e:
            print(f"Erro na análise dos instrumentos: {str(e)}")
            return {}

    def _benchmark_comparison(self, data):
        """
        Compara resultados com valores de referência da literatura.
        """
        comparison = {}

        try:
            # Intervalos típicos baseados na literatura
            labor_elasticity = self.production_params[data['setor_ipa_7d'].iloc[0]]['variable_input']
            returns_to_scale = sum(self.production_params[data['setor_ipa_7d'].iloc[0]].values())

            comparison['labor_elasticity_ok'] = 0.2 <= labor_elasticity <= 0.8
            comparison['returns_to_scale_ok'] = 0.8 <= returns_to_scale <= 1.2

            return comparison

        except Exception as e:
            print(f"Erro na comparação com benchmarks: {str(e)}")
            return {}

    def _calculate_vif(self, X):
        """
        Calcula o Fator de Inflação da Variância (VIF) para detectar
        multicolinearidade entre as variáveis explicativas.
        """
        vif_data = pd.DataFrame()
        vif_data["Variável"] = X.columns

        # Calcular VIF para cada variável
        vif_values = []
        for i in range(X.shape[1]):
            y = X.iloc[:, i]
            X_others = X.drop(X.columns[i], axis=1)
            r2 = sm.OLS(y, sm.add_constant(X_others)).fit().rsquared
            vif = 1/(1 - r2)
            vif_values.append(vif)

        vif_data["VIF"] = vif_values
        return vif_data
             
    def save_markup_summary(self, results, output_dir, df_prep):
        """
        Salva estatísticas resumidas dos markups por setor.
        
        Args:
            results: Dicionário com resultados de estimação
            output_dir: Diretório de saída
            df_prep: DataFrame preparado (mantido por consistência com outras funções)
        """
        try:
            Path(output_dir).mkdir(parents=True, exist_ok=True)
            
            # Lista para armazenar dados de resumo
            summary_data = []
            
            for sector, res in results.items():
                if 'markups' not in res or len(res['markups']) == 0:
                    print(f"Aviso: Sem markups para o setor {sector}. Pulando.")
                    continue
                
                try:
                    # Obter markups válidos
                    markups = res['markups']
                    valid_markups = markups[~np.isnan(markups)]
                    
                    if len(valid_markups) == 0:
                        continue
                        
                    # Calcular estatísticas resumidas
                    summary_stats = {
                        'setor_codigo': sector,
                        'setor_nome': res.get('nome_setor', sector),
                        'mean': np.mean(valid_markups),
                        'std': np.std(valid_markups),
                        'median': np.median(valid_markups),
                        'min': np.min(valid_markups),
                        'max': np.max(valid_markups),
                        'q25': np.percentile(valid_markups, 25),
                        'q75': np.percentile(valid_markups, 75),
                        'skewness': stats.skew(valid_markups),
                        'kurtosis': stats.kurtosis(valid_markups),
                        'count': len(valid_markups)
                    }
                    
                    summary_data.append(summary_stats)
                    
                except Exception as sector_error:
                    print(f"Erro no processamento do resumo para o setor {sector}: {sector_error}")
            
            # Salvar dados de resumo
            if summary_data:
                summary_df = pd.DataFrame(summary_data)
                summary_df = summary_df.round(4)
                
                summary_df.to_excel(
                    Path(output_dir) / 'markup_sumario.xlsx',
                    index=False
                )
                
                print(f"Resumo de markups salvo. Total de setores: {len(summary_df)}")
            else:
                print("Nenhum dado de resumo de markup válido para salvar.")
                
        except Exception as e:
            print(f"Erro ao salvar resumo de markups: {str(e)}")
        
    def save_productivity_results(self, results, output_dir, df_prep):
        """
        Save productivity estimates in a structured format with separate firm-level
        and summary files.

        Args:
            results: Dictionary containing estimation results
            output_dir: Output directory path
            df_prep: Prepared DataFrame with firm/time identifiers
        """
        try:
            Path(output_dir).mkdir(parents=True, exist_ok=True)

            # List to store productivity results
            all_productivity = []
            summary_data = []

            for sector, res in results.items():
                if 'productivity' not in res or len(res['productivity']) == 0:
                    print(f"Warning: No productivity estimates for sector {sector}. Skipping.")
                    continue

                try:
                    # Select sector data
                    sector_data = df_prep[df_prep['setor_ipa_7d'] == sector].copy()

                    # Get productivity values
                    productivity = res['productivity'][:len(sector_data)]

                    # Ensure lengths match
                    if len(productivity) != len(sector_data):
                        print(f"Adjusting productivity array length for sector {sector}")
                        productivity = np.pad(productivity, 
                                           (0, len(sector_data) - len(productivity)),
                                           mode='constant', 
                                           constant_values=np.nan)

                    # Create firm-level results
                    sector_productivity = pd.DataFrame({
                        'setor_codigo': sector,
                        'setor_nome': res.get('nome_setor', sector),
                        'empresa': sector_data['empresa'],
                        'ano': sector_data['ano'],
                        'trimestre': sector_data['trimestre'],
                        'produtividade': productivity
                    })

                    all_productivity.append(sector_productivity)

                    # Calculate sector summary statistics
                    valid_prod = productivity[~np.isnan(productivity)]
                    summary_stats = {
                        'setor_codigo': sector,
                        'setor_nome': res.get('nome_setor', sector),
                        'mean': np.mean(valid_prod),
                        'std': np.std(valid_prod),
                        'median': np.median(valid_prod),
                        'min': np.min(valid_prod),
                        'max': np.max(valid_prod),
                        'q25': np.percentile(valid_prod, 25),
                        'q75': np.percentile(valid_prod, 75),
                        'skewness': stats.skew(valid_prod),
                        'kurtosis': stats.kurtosis(valid_prod),
                        'count': len(valid_prod)
                    }

                    summary_data.append(summary_stats)

                except Exception as sector_error:
                    print(f"Error processing sector {sector}: {sector_error}")

            # Save firm-level data
            if all_productivity:
                productivity_df = pd.concat(all_productivity, ignore_index=True)
                productivity_df = productivity_df.dropna(subset=['produtividade'])

                productivity_df.to_excel(
                    Path(output_dir) / 'produtividade_empresa.xlsx', 
                    index=False
                )

                print(f"Firm-level productivity saved. Total rows: {len(productivity_df)}")

            # Save summary statistics
            if summary_data:
                summary_df = pd.DataFrame(summary_data)
                summary_df = summary_df.round(4)

                summary_df.to_excel(
                    Path(output_dir) / 'produtividade_sumario.xlsx',
                    index=False
                )

                print(f"Summary statistics saved. Total sectors: {len(summary_df)}")

        except Exception as e:
            print(f"Fatal error saving productivity results: {str(e)}")
    
    def save_estimation_results(self, results, output_dir, df_prep):
        try:
            Path(output_dir).mkdir(parents=True, exist_ok=True)

            # Lista para armazenar resultados de todos os setores
            all_results = []

            for sector, res in results.items():
                if 'markups' not in res or len(res['markups']) == 0:
                    print(f"Aviso: Sem markups para o setor {sector}. Pulando.")
                    continue

                try:
                    # Selecionar dados do setor
                    sector_data = df_prep[df_prep['setor_ipa_7d'] == sector].copy()

                    # Truncar ou expandir markups para corresponder ao tamanho de sector_data
                    markups = res['markups'][:len(sector_data)]

                    # Garantir que os tamanhos correspondam
                    if len(markups) != len(sector_data):
                        print(f"Ajustando tamanho dos markups para o setor {sector}")
                        markups = np.pad(markups, (0, len(sector_data) - len(markups)), 
                                         mode='constant', constant_values=np.nan)

                    sector_results = pd.DataFrame({
                        'setor_codigo': sector,
                        'setor_nome': res.get('nome_setor', sector),
                        'empresa': sector_data['empresa'],
                        'ano': sector_data['ano'],
                        'trimestre': sector_data['trimestre'], 
                        'markup': markups
                    })

                    all_results.append(sector_results)
                   

                except Exception as sector_error:
                    print(f"Erro no processamento do setor {sector}: {sector_error}")

            # Combinar resultados de todos os setores
            if all_results:
                results_df = pd.concat(all_results, ignore_index=True)

                # Remover linhas com markups NaN, se necessário
                results_df = results_df.dropna(subset=['markup'])

                # Salvar arquivo
                results_df.to_excel(Path(output_dir) / 'markups_empresa.xlsx', index=False)
                # Add this line to save diagnostic results
                self.save_diagnostic_results(results, output_dir)
                print(f"Resultados salvos. Total de linhas: {len(results_df)}")
            else:
                print("Nenhum resultado válido para salvar.")

        except Exception as e:
            print(f"Erro fatal ao salvar resultados: {str(e)}")
            
    def save_diagnostic_results(self, results, output_dir):
        try:
            Path(output_dir).mkdir(parents=True, exist_ok=True)

            # Prepare comprehensive diagnostic DataFrame
            diagnostic_data = []

            for sector, res in results.items():
                try:
                    # Safe extraction of markups with error handling
                    markups = res.get('markups', [])
                    markups = [markup] if isinstance(markups, (int, float)) else markups
                    markups = [markup for markup in markups if pd.notnull(markup)]

                    sector_diagnostics = {
                        'setor_codigo': sector,
                        'setor_nome': res.get('nome_setor', 'N/A'),

                        # First Stage Diagnostics
                        'first_stage_r2': self.diagnostics.get(sector, {}).get('first_stage', {}).get('model', {}).get('r2', np.nan),
                        'first_stage_r2_adj': self.diagnostics.get(sector, {}).get('first_stage', {}).get('model', {}).get('r2_adj', np.nan),
                        'productivity_mean': self.diagnostics.get(sector, {}).get('first_stage', {}).get('productivity', {}).get('mean', np.nan),
                        'productivity_std': self.diagnostics.get(sector, {}).get('first_stage', {}).get('productivity', {}).get('std', np.nan),

                        # Second Stage Diagnostics
                        'second_stage_r2': self.diagnostics.get(sector, {}).get('second_stage', {}).get('second_stage_r2', np.nan),
                        'returns_to_scale': res.get('second_stage', {}).get('returns_to_scale', np.nan),

                        # Elasticity Information
                        'variable_input_elasticity': res.get('elasticities', {}).get('variable_input', np.nan),
                        'capital_elasticity': res.get('elasticities', {}).get('capital', np.nan),

                        # Markup Statistics
                        'markup_mean': float(np.mean(markups)) if markups else np.nan,
                        'markup_median': float(np.median(markups)) if markups else np.nan,
                        'markup_std': float(np.std(markups)) if markups else np.nan,

                        # Additional Metadata
                        'n_observations': len(markups),
                    }

                    diagnostic_data.append(sector_diagnostics)

                except Exception as sector_error:
                    print(f"Error processing diagnostics for sector {sector}: {sector_error}")

            # Convert to DataFrame
            diagnostics_df = pd.DataFrame(diagnostic_data)

            # Save to Excel
            diagnostics_path = Path(output_dir) / 'diagnosticos.xlsx'
            diagnostics_df.to_excel(diagnostics_path, index=False)

            print(f"Diagnostic results saved. Total sectors: {len(diagnostics_df)}")

        except Exception as e:
            print(f"Error saving diagnostic results: {str(e)}")

if __name__ == "__main__":
    # Definir caminhos
    input_path = Path(r"C:\Users\fabio\OneDrive\Projeto_Markup\Results\Model2\v2\final_data.xlsx")
    output_dir = Path(r"C:\Users\fabio\OneDrive\Projeto_Markup\Results\Model2\v2\Estimacao")
        
    # Carregar dados
    print("Carregando dados...")
    df = pd.read_excel(input_path)
        
    # Inicializar estimador  
    estimator = TwoInputMarkupEstimator(polynomial_degree=3)
        
    # Preparar dados
    df_prep = estimator.prepare_data(df)
        
    results = {}
        
    # Processar cada setor
    for sector in df_prep['setor_ipa_7d'].unique():
        if pd.isna(sector):
            continue
                
        sector_name = df_prep[df_prep['setor_ipa_7d'] == sector]['nome_setor'].iloc[0] 
        print(f"\nProcessando setor: {sector_name}")
            
        # Estimar função de produção
        sector_results = estimator.estimate_production_function(df_prep, sector)
            
        if sector_results is None:
            continue
            
        # Calcular markups
        markups = estimator.calculate_markups(df_prep, sector)
            
        # Armazenar resultados
        results[sector] = {
            **sector_results,
            'markups': markups,  
            'nome_setor': sector_name
        }
            
        # Realizar análise de sensibilidade
        sensitivity = estimator.sensitivity_analysis(df_prep, sector, markups)
        if sensitivity:
            results[sector]['sensitivity'] = sensitivity

    # Salvar resultados da estimação  
    estimator.save_estimation_results(results, output_dir, df_prep)
    
    # Add this line to save productivity results
    estimator.save_productivity_results(results, output_dir, df_prep)

    # Save markup results
    estimator.save_markup_summary(results, output_dir, df_prep)

Carregando dados...
Preparando dados para estimação...
  Removidas 58 observações com valores não-positivos em variable_input
  Removidas 580 observações com valores não-positivos em capital
  Removidas 1600 observações com valores não-positivos em receita

Observações válidas: 8326/10360 (80.4%)

Processando setor: Alimentos e bebidas

Estimando função de produção para setor: Alimentos e bebidas

Primeiro estágio da estimação - Setor: 2022100

Diagnósticos do Primeiro Estágio:
Média da produtividade: -0.0000
Desvio padrão: 0.1288
Correlação com insumo variável: -0.0000
R² da forma reduzida: 0.9974
Número de termos significativos: 7

Segundo estágio da estimação - Setor: 2022100

Diagnósticos do Segundo Estágio:
F-stat primeiro estágio: 57406.52
R² primeiro estágio: 0.9948
R² segundo estágio: 0.8702
Retornos de escala: 0.8546
Elasticidade do insumo variável: 0.6040
Elasticidade do capital: 0.2506

Estimando função de produção para setor: Alimentos e bebidas

Primeiro estágio da estimaç